# CA2 - Assessment Task

Online retailer, eBay is providing an option of bidding to their customers globally. Bidding is employed to find the real price of items in the market based on the demand. The price offered by anyone participating in this process is termed as a 'bid'. A dataset is available at the following link and on Moodle for the bidding of customers

Normal bids are classified as `0` bids in the data set and anomalous bids as `1`. Your goal is to **use classification or clustering algorithms to predict the bids in the future**. You would need to clean and prepare the dataset for the machine learning modelling under the following guidelines. 

* Provide a logical justification based on the reasoning for the specific choice of machine learning approaches (supervised/unsupervised) for the provided Shill bidding data set

* Machine Learning models can be used for Prediction, Classification, Clustering and Time series analysis. You should plan on trying multiple models (at least two) with proper parameter-selection using hyperparameters and show a comparison between the chosen modelling approaches.

* You should train and test the Machine learning models in the case of supervised learning and use other metrics for unsupervised learning if appropriate. Use cross validation to provide authenticity of the modelling outcomes. Perform a comparison of ML modelling outcomes using a Table or graph visualization.

* Describe the rationale for the selection of models and justify the choice of hyperparameters.

* A Table or graphics should be provided to illustrate the similarities and contrast of the Machine Learning modelling outcomes

### References 

https://rstudio-pubs-static.s3.amazonaws.com/938731_0a06285936be4176ac1bd2da0955f499.html

### Import / Configuration

In [8]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

import pandas_profiling
from pandas_profiling import ProfileReport

## Data Understanding

In [3]:
df = pd.read_csv('Shill Bidding Dataset.csv')

,Record_ID,Auction_ID,Bidder_ID,Bidder_Tendency,Bidding_Ratio,Successive_Outbidding,Last_Bidding,Auction_Bids,Starting_Price_Average,Early_Bidding,Winning_Ratio,Auction_Duration,Class
0,1,732,_***i,0.200000,0.400000,0.0,0.000028,0.000000,0.993593,0.000028,0.666667,5,0
1,2,732,g***r,0.024390,0.200000,0.0,0.013123,0.000000,0.993593,0.013123,0.944444,5,0
2,3,732,t***p,0.142857,0.200000,0.0,0.003042,0.000000,0.993593,0.003042,1.000000,5,0
3,4,732,7***n,0.100000,0.200000,0.0,0.097477,0.000000,0.993593,0.097477,1.000000,5,0
4,5,900,z***z,0.051282,0.222222,0.0,0.001318,0.000000,0.000000,0.001242,0.500000,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6316,15129,760,l***t,0.333333,0.160000,1.0,0.738557,0.280000,0.993593,0.686358,0.888889,3,1
6317,15137,2481,s***s,0.030612,0.130435,0.0,0.005754,0.217391,0.993593,0.000010,0.878788,7,0
6318,15138,2481,h***t,0.055556,0.043478,0.0,0.015663,0.217391,0.993593,0.015663,0.000000,7,0
6319,15139,2481,d***d,0.076923,0.086957,0.0,0.068694,0.217391,0.993593,0.000415,0.000000,7,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6321 entries, 0 to 6320
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Record_ID               6321 non-null   int64  
 1   Auction_ID              6321 non-null   int64  
 2   Bidder_ID               6321 non-null   object 
 3   Bidder_Tendency         6321 non-null   float64
 4   Bidding_Ratio           6321 non-null   float64
 5   Successive_Outbidding   6321 non-null   float64
 6   Last_Bidding            6321 non-null   float64
 7   Auction_Bids            6321 non-null   float64
 8   Starting_Price_Average  6321 non-null   float64
 9   Early_Bidding           6321 non-null   float64
 10  Winning_Ratio           6321 non-null   float64
 11  Auction_Duration        6321 non-null   int64  
 12  Class                   6321 non-null   int64  
dtypes: float64(8), int64(4), object(1)
memory usage: 642.1+ KB


In [9]:
prof = ProfileReport(df)
prof.to_file(output_file='pdp_report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

[Click Here](pdp_report.html)

## Data Preparation

## Modeling

## Evaluation